# Modifying the mLAMA dataset with full-sentence translation

This notebook shows a cycle of curating one language-relation pair within the mLAMA dataset. With the current implementation (that does not feature access to the paid Google Translate API), you would have to do every language-relation pair by hand. However, I feel that there is a positive side in it: you can see the translation artifacts as they emerge.

## 1. Opening the dataset

Import the class and specify the language and relation of your interest. Also, specify the path with the initial MLAMA dataset (you can download it from here: https://github.com/norakassner/mlama 

In [1]:
from datamodifier import DataModifier

/opt/anaconda3/envs/try-mps/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
lang = 'ru'
rel = 'P19'

dm = DataModifier('../mlama/data/mlama1.1/', lang, rel, trial=False)

In [31]:
dm.df.tail(20)

,sub_uri,en_sub_label,sub_label,obj_uri,en_obj_label,obj_label,en_template,template,template_prompt
142,Q6337245,KRU,KRU,Q183412,EMI,EMI,KRU is represented by music label EMI .,KRU je reprezentován hudební značkou EMI.,KRU je reprezentován hudební značkou
143,Q46087,Anouk,Anouk,Q183412,EMI,EMI,Anouk is represented by music label EMI .,Anouk je reprezentován hudební značkou EMI.,Anouk je reprezentován hudební značkou
144,Q26933,Pete Townshend,Pete Townshend,Q557632,Decca,Decca Records,Pete Townshend is represented by music label D...,Pete Townshend je reprezentován hudební značko...,Pete Townshend je reprezentován hudební značkou
145,Q44209,Music & Me,Music & Me,Q43327,Motown,Motown Records,Music & Me is represented by music label Motown .,Music & Me je reprezentován hudební značkou Mo...,Music & Me je reprezentován hudební značkou
146,Q82545,Sex Pistols,Sex Pistols,Q183412,EMI,EMI,Sex Pistols is represented by music label EMI .,Sex Pistols je reprezentován hudební značkou EMI.,Sex Pistols je reprezentován hudební značkou
147,Q1330456,Takk...,Takk...,Q183412,EMI,EMI,Takk... is represented by music label EMI .,Takk... je reprezentován hudební značkou EMI.,Takk... je reprezentován hudební značkou
148,Q355788,Roy Smeck,Roy Smeck,Q557632,Decca,Decca Records,Roy Smeck is represented by music label Decca .,Roy Smeck je reprezentován hudební značkou Dec...,Roy Smeck je reprezentován hudební značkou
149,Q229330,Vera Lynn,Vera Lynn,Q557632,Decca,Decca Records,Vera Lynn is represented by music label Decca .,Vera Lynn je reprezentován hudební značkou Dec...,Vera Lynn je reprezentován hudební značkou
150,Q371795,Riccardo Chailly,Riccardo Chailly,Q557632,Decca,Decca Records,Riccardo Chailly is represented by music label...,Riccardo Chailly je reprezentován hudební znač...,Riccardo Chailly je reprezentován hudební znač...
151,Q201908,Anni-Frid Lyngstad,Anni-Frid Lyngstadová,Q183412,EMI,EMI,Anni-Frid Lyngstad is represented by music lab...,Anni-Frid Lyngstadová je reprezentován hudební...,Anni-Frid Lyngstadová je reprezentován hudební...


# Creating new columns

### Temporary workaround for GoogleTranslate:

This is the only thing you'd have to do **by hands**. I did not use the Google Translate API (for the price reasons), and the free API workarounds downgrade the version of the NMT and therefore give not the best translations. Therefore, follow the next steps: 

- take the english sentences from an xlsx file from `gt_doc/src` folder (it's already saved)
- translate them through the document translation in Google web interface
- upload them to `gt_doc/tgt` folder 
- change the column name to "gt_translation"

In [33]:
dm.add_google_translations(mode='manual_doc')
#dm.add_google_translations(mode='workaround', aux_path='./temp_gt/cs_P103.csv')

### Temporary workaround for chatgpt translation


#### Step 1: create few-shot examples 
This is an optional step for Experiment 1 (comparing template translation against both Google Translate and ChatGPT few-shot translation). You need to create 20-shot examples. I sampled the examples from the pool of entities; you can come up with your examples; but you need to adhere to the format of the few-shot file that is being created in the next 

- make sure google translation is there 
- run the `get_sample_for_chatgpt_fewshot` below to get the sample for the few-shot sentences; try to make the labels as different as possible
- check all translations manually! Pay special attention to the adherence of the named entity translation
- save the file to `chatgpt_prompts/translation20` folder in the naming format: `{lang_id}_{rel_id}.txt`, for example, `cs_P19.txt`

In [34]:
id2lang = {'cs': 'Czech', 'uk': 'Ukrainian', 'ru': 'Russian', 'hr': 'Croatian'}

print(dm.get_sample_for_chatgpt_fewshot(n=20, tgt_lang=id2lang[lang], use_translation=True))


        You are a professional English-Czech translator. 
        You are given the English sentences about subjects and objects. You are also given translations of subjects and objects separately. 
        You need to translate full sentences to Czech. When translating, you have to use the translated subjects and objects. 
        Pay special attention to grammatical agreement between the words in the translated sentences.
        When translating, follow the examples:
        
        Source sentence: Riccardo Chailly is represented by music label Decca .
        Subject translation: Riccardo Chailly
        Object translation: Decca Records
        Translation:  Riccardo Chailly je zastupován hudebním vydavatelstvím Decca.
        
        Source sentence: Tin Machine is represented by music label EMI .
        Subject translation: Tin Machine
        Object translation: EMI
        Translation:  Tin Machine je zastoupena hudební značkou EMI.
        
        Source sentence: Stevi

#### Step 2: run the ChatGPT translation

**NB** for that you'd need an OpenAI API key! 

In [ ]:

# dm.df.to_csv(f'intermediate_data/{lang}_{rel}.tsv', sep='\t')
dm.add_chatgpt_translation(system_prompt_folder='./chatgpt_prompts/translation20/')

In [4]:
dm.add_chatgpt_translation(system_prompt_folder='./chatgpt_prompts/translation20/')

        You are a professional English-Russian translator. 
        You are given the English sentences about subjects and objects. You are also given translations of subjects and objects separately. 
        You need to translate full sentences to Russian. When translating, you have to use the translated subjects and objects. 
        Pay special attention to grammatical agreement between the words in the translated sentences.
        When translating, follow the examples:
        
        Source sentence: Max Weber works in the field of history .
        Subject translation: Макс Вебер
        Object translation: история
        Translation: Макс Вебер работает в области истории.
        
        Source sentence: Steve Biko works in the field of apartheid .
        Subject translation: Стив Бико
        Object translation: апартеид
        Translation: Стив Бико работает в области апартеида.
        
        Source sentence: Ivan Sechenov works in the field of physiology .
        Su

## Automatic processing: 

### Adding splits between prompt core ("London is a capital of..." and objects "...The UK")

for some relation types (for example P103, native language) the repetitive formulation of the object entity in Wikidata is "X language" (for example, "Russian language"); while in Google and ChatGPT translation the phrases would be "...'s native language is Russian." Currently, the model would not be able to identify such a word. To overcome it, use the `cutoff` parameter to ignore the words like "language" and split by really important entities

In [1]:
dm.add_prompt_obj_split()

# example for cutoff usage: 
# dm.add_prompt_obj_split(cutoff=': язык')

NameError: name 'dm' is not defined

### Retrieving object aliases from wikidata

**NB**: after the publication of the paper, the interface of the Wikidata API changed. Please refer to these fixes: https://github.com/dahlia/wikidata/issues/64 

In [6]:
dm.add_aliases()

## Saving the updated dataset to the folder

* if you used ChatGPT, put the `comment` parameter a `20prompt` value
* if there was only Google Translate, put a `nogpt` value

In [10]:
dm.save_df(comment='20prompt')

In [9]:
dm.df.tail(20)

,sub_uri,en_sub_label,sub_label,obj_uri,en_obj_label,obj_label,en_template,template,template_prompt,gt_translation,chatgpt_translation,gt_prompt,gt_infl_object,chatgpt_prompt,chatgpt_infl_object,aliases
182,Q361405,Theodore the Studite,Теодор Студит,Q16869,Constantinople,Константинополь,Theodore the Studite was born in Constantinople .,Теодор Студит народився в Константинополь.,Теодор Студит народився в,Феодор Студит народився в Константинополі.,Теодор Студит народився в Константинополі.,Феодор Студит народився в,Константинополі,Теодор Студит народився в,Константинополі,Constantinople|||Костянтинополь|||Царгород
183,Q65239,Hermann Dessau,Герман Дессау,Q1794,Frankfurt am Main,Франкфурт-на-Майні,Hermann Dessau was born in Frankfurt am Main .,Герман Дессау народився в Франкфурт-на-Майні.,Герман Дессау народився в,Герман Дессау народився у Франкфурті-на-Майні.,Герман Дессау народився у Франкфурті-на-Майні.,Герман Дессау народився у,Франкфурті-на-Майні,Герман Дессау народився у,Франкфурті-на-Майні,Frankfurt|||Франкфурт
184,Q162543,Mehmed VI,Мехмед VI,Q406,Istanbul,Стамбул,Mehmed VI was born in Istanbul .,Мехмед VI народився в Стамбул.,Мехмед VI народився в,Мехмед VI народився в Стамбулі.,Мехмед VI народився в Стамбулі.,Мехмед VI народився в,Стамбулі,Мехмед VI народився в,Стамбулі,Istanbul
185,Q352935,Dennis Farina,Денніс Фаріна,Q1297,Chicago,Чикаго,Dennis Farina was born in Chicago .,Денніс Фаріна народився в Чикаго.,Денніс Фаріна народився в,Денніс Фаріна народився в Чикаго.,Денніс Фаріна народився в Чикаго.,Денніс Фаріна народився в,Чикаго,Денніс Фаріна народився в,Чикаго,"Chicago|||Чикаго, Іллінойс|||Чікаґо"
186,Q110916,Hideki Shirakawa,Хідекі Сіракава,Q1490,Tokyo,Токіо,Hideki Shirakawa was born in Tokyo .,Хідекі Сіракава народився в Токіо.,Хідекі Сіракава народився в,Хідекі Сіракава народився в Токіо.,Хідекі Сіракава народився в Токіо.,Хідекі Сіракава народився в,Токіо,Хідекі Сіракава народився в,Токіо,Tokyo|||Токіо метрополіс|||Префектура Токіо|||...
187,Q313063,Brendan Behan,Біен Брендан,Q1761,Dublin,Дублін,Brendan Behan was born in Dublin .,Біен Брендан народився в Дублін.,Біен Брендан народився в,Брендан Бехан народився в Дубліні.,Біен Брендан народився в Дубліні.,Брендан Бехан народився в,Дубліні,Біен Брендан народився в,Дубліні,Dublin|||Деблін
188,Q160451,Luigi Nono,Луїджі Ноно,Q641,Venice,Венеція,Luigi Nono was born in Venice .,Луїджі Ноно народився в Венеція.,Луїджі Ноно народився в,Луїджі Ноно народився у Венеції.,Луїджі Ноно народився у Венеції.,Луїджі Ноно народився у,Венеції,Луїджі Ноно народився у,Венеції,Venice
189,Q4430657,Suren Spandaryan,Спандарян Сурен Спандарович,Q994,Tbilisi,Тбілісі,Suren Spandaryan was born in Tbilisi .,Спандарян Сурен Спандарович народився в Тбілісі.,Спандарян Сурен Спандарович народився в,Сурен Спандарян народився в Тбілісі.,Спандарян Сурен Спандарович народився в Тбілісі.,Сурен Спандарян народився в,Тбілісі,Спандарян Сурен Спандарович народився в,Тбілісі,Tbilisi|||Тифліс
190,Q311820,Wat Tyler,Вот Тайлер,Q23298,Kent,Кент,Wat Tyler was born in Kent .,Вот Тайлер народився в Кент.,Вот Тайлер народився в,Уот Тайлер народився в Кенті.,Вот Тайлер народився в Кенті.,Уот Тайлер народився в,Кенті,Вот Тайлер народився в,Кенті,Kent|||графство Кент|||церемоніальне графство ...
191,Q447963,Friedrich August von Kaulbach,Фрідріх Август фон Каульбах,Q1726,Munich,Мюнхен,Friedrich August von Kaulbach was born in Muni...,Фрідріх Август фон Каульбах народився в Мюнхен.,Фрідріх Август фон Каульбах народився в,Фрідріх Август фон Каульбах народився в Мюнхені.,Фрідріх Август фон Каульбах народився в Мюнхені.,Фрідріх Август фон Каульбах народився в,Мюнхені,Фрідріх Август фон Каульбах народився в,Мюнхені,Munich
